## MEDQUAD DATA

In [1]:
import torch

In [64]:
import os
RAW_MEDQUAD_DIR = '../data/raw/medquad/'
os.listdir(RAW_MEDQUAD_DIR)

['3_GHR_QA',
 '2_GARD_QA',
 '6_NINDS_QA',
 '.DS_Store',
 '12_MPlusHerbsSupplements_QA',
 '1_CancerGov_QA',
 '8_NHLBI_QA_XML',
 'QA-TestSet-LiveQA-Med-Qrels-2479-Answers.zip',
 '4_MPlus_Health_Topics_QA',
 '7_SeniorHealth_QA',
 '10_MPlus_ADAM_QA',
 '9_CDC_QA',
 'readme.txt',
 '.git',
 'LICENSE.txt',
 '5_NIDDK_QA',
 '11_MPlusDrugs_QA']

In [94]:
l = glob.glob(RAW_MEDQUAD_DIR+'/3_GHR_QA/*.xml')

l[3000:]

[]

In [97]:
import glob

l =glob.glob(RAW_MEDQUAD_DIR+'/*/*.xml',recursive=True)
l[3000:]

['../data/raw/medquad/2_GARD_QA/0003201.xml',
 '../data/raw/medquad/2_GARD_QA/0005116.xml',
 '../data/raw/medquad/2_GARD_QA/0002679.xml',
 '../data/raw/medquad/2_GARD_QA/0001370.xml',
 '../data/raw/medquad/2_GARD_QA/0006580.xml',
 '../data/raw/medquad/2_GARD_QA/0004397.xml',
 '../data/raw/medquad/2_GARD_QA/0002080.xml',
 '../data/raw/medquad/2_GARD_QA/0000683.xml',
 '../data/raw/medquad/2_GARD_QA/0004383.xml',
 '../data/raw/medquad/2_GARD_QA/0002725.xml',
 '../data/raw/medquad/2_GARD_QA/0006543.xml',
 '../data/raw/medquad/2_GARD_QA/0002043.xml',
 '../data/raw/medquad/2_GARD_QA/0003349.xml',
 '../data/raw/medquad/2_GARD_QA/0002719.xml',
 '../data/raw/medquad/2_GARD_QA/0003407.xml',
 '../data/raw/medquad/2_GARD_QA/0003361.xml',
 '../data/raw/medquad/2_GARD_QA/0001576.xml',
 '../data/raw/medquad/2_GARD_QA/0003375.xml',
 '../data/raw/medquad/2_GARD_QA/0005704.xml',
 '../data/raw/medquad/2_GARD_QA/0003413.xml',
 '../data/raw/medquad/2_GARD_QA/0000495.xml',
 '../data/raw/medquad/2_GARD_QA/00

In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
import json

def extract_one_medquad_file(xml_file):
    """
    Parses a single MedQuAD XML where <Document> is the root element.
    Extracts synonyms, semantic group, question, and answer.

    Args:
        xml_file (str): Path to the MedQuAD XML file.

    Returns:
        List[dict]: A list of Q&A dictionaries containing relevant fields.
    """

    # Parse the XML
    tree = ET.parse(xml_file)
    root = tree.getroot()  # <Document> is the root in your snippet

    # Basic Document-level attributes
    doc_id = root.get("id", "")
    source = root.get("source", "")
    url = root.get("url", "")

    # Focus: Main topic/disease name
    focus = root.findtext("Focus", default="")

    # Extract synonyms
    synonyms = []
    syn_path = root.find("FocusAnnotations/Synonyms")
    if syn_path is not None:
        for syn_el in syn_path.findall("Synonym"):
            if syn_el.text:
                synonyms.append(syn_el.text.strip())

    # Extract semantic group
    sem_group = None
    sem_group_el = root.find("FocusAnnotations/UMLS/SemanticGroup")
    if sem_group_el is not None and sem_group_el.text:
        sem_group = sem_group_el.text.strip()

    # Collect Q&A pairs
    qa_data = []
    qapairs = root.find("QAPairs")
    if qapairs is not None:
        for pair in qapairs.findall("QAPair"):
            pid = pair.get("pid", "")
            question_el = pair.find("Question")
            answer_el = pair.find("Answer")

            question = question_el.text if question_el is not None else ""
            qtype = question_el.get("qtype") if question_el is not None else ""
            answer = answer_el.text if answer_el is not None else ""

            question =  question and question.strip()
            answer = answer  and answer.strip()

            # Build a record for each Q&A
            qa_data.append({
                "document_id": doc_id,
                "source": source,
                "url": url,
                "focus": focus,
                "synonyms": ','.join(synonyms) if synonyms else '',
                "semantic_group": sem_group,
                "pid": pid,
                "qtype": qtype,
                "question": question,
                "answer": answer
            })

    return qa_data



# if __name__ == "__main__":
#     # Example usage with a single MedQuAD XML file
#     xml_file_path = "data/raw/medquad_sample.xml"  # Adjust path as needed
#     records = extract_one_medquad_file(xml_file_path)

#     # Convert to DataFrame
#     df = pd.DataFrame(records)

#     # Save to CSV
#     csv_out = "data/processed/medquad_parsed.csv"
#     df.to_csv(csv_out, index=False, encoding="utf-8")
#     print(f"CSV saved to {csv_out}")

#     # Or save to JSON
#     json_out = "data/processed/medquad_parsed.json"
#     with open(json_out, "w", encoding="utf-8") as f:
#         json.dump(records, f, indent=4, ensure_ascii=False)
#     print(f"JSON saved to {json_out}")

In [128]:
import os, json
import pandas as pd


def extract_medquad(xml_files):
    # Initialize storage lists
    all_records = []
    for file in xml_files:
        one_record = extract_one_medquad_file(file)
        all_records.extend(one_record)

    df = pd.DataFrame(all_records)

    return df

all_xml_files = glob.glob('../data/raw/medquad/*/*.xml', recursive=True)
print(len(all_xml_files))

# if __name__ == '__main__':
df_medquad = extract_medquad(all_xml_files)
df_medquad.to_csv("../data/processed/medquad_extracted.csv", index=False)

5497


In [129]:
df_medquad

,document_id,source,url,focus,synonyms,semantic_group,pid,qtype,question,answer
0,0000559,GHR,https://ghr.nlm.nih.gov/condition/keratoderma-...,keratoderma with woolly hair,KWWH,Disorders,1,information,What is (are) keratoderma with woolly hair ?,Keratoderma with woolly hair is a group of rel...
1,0000559,GHR,https://ghr.nlm.nih.gov/condition/keratoderma-...,keratoderma with woolly hair,KWWH,Disorders,2,frequency,How many people are affected by keratoderma wi...,Keratoderma with woolly hair is rare; its prev...
2,0000559,GHR,https://ghr.nlm.nih.gov/condition/keratoderma-...,keratoderma with woolly hair,KWWH,Disorders,3,genetic changes,What are the genetic changes related to kerato...,"Mutations in the JUP, DSP, DSC2, and KANK2 gen..."
3,0000559,GHR,https://ghr.nlm.nih.gov/condition/keratoderma-...,keratoderma with woolly hair,KWWH,Disorders,4,inheritance,Is keratoderma with woolly hair inherited ?,Most cases of keratoderma with woolly hair hav...
4,0000559,GHR,https://ghr.nlm.nih.gov/condition/keratoderma-...,keratoderma with woolly hair,KWWH,Disorders,5,treatment,What are the treatments for keratoderma with w...,These resources address the diagnosis or manag...
...,...,...,...,...,...,...,...,...,...,...
16407,0000193,NIDDK,http://www.niddk.nih.gov/health-information/he...,What I need to know about Cirrhosis,,Disorders,5,treatment,What are the treatments for What I need to kno...,"Once you have cirrhosis, nothing can make all ..."
16408,0000193,NIDDK,http://www.niddk.nih.gov/health-information/he...,What I need to know about Cirrhosis,,Disorders,6,treatment,What are the treatments for What I need to kno...,"If too much scar tissue forms, your liver coul..."
16409,0000193,NIDDK,http://www.niddk.nih.gov/health-information/he...,What I need to know about Cirrhosis,,Disorders,7,prevention,How to prevent What I need to know about Cirrh...,"To prevent cirrhosis,\n \n- see..."
16410,0000193,NIDDK,http://www.niddk.nih.gov/health-information/he...,What I need to know about Cirrhosis,,Disorders,9,considerations,What to do for What I need to know about Cirrh...,- Cirrhosis is scarring of the liver. Scar tis...


In [147]:
df_medquad.qtype.unique()

array(['information', 'frequency', 'genetic changes', 'inheritance',
       'treatment', 'symptoms', 'causes', 'exams and tests', 'outlook',
       'research', 'complications', 'susceptibility', 'stages',
       'prevention', 'considerations', 'support groups'], dtype=object)

In [135]:
df_medquad.semantic_group.unique()

array(['Disorders', None], dtype=object)

In [142]:
cols = df_medquad.columns

In [149]:
for col, dat in zip(cols, df_medquad.iloc[15000]):
    print(col ,':', dat)


document_id : 0000212
source : CDC
url : http://www.cdc.gov/hantavirus/
focus : Hantavirus
synonyms : 
semantic_group : None
pid : 4
qtype : prevention
question : how can hps be prevented for Hantavirus ?
answer : Eliminate or minimize contact with rodents in your home, workplace, or campsite. If rodents don't find that where you are is a good place for them to be, then you're less likely to come into contact with them. Seal up holes and gaps in your home or garage. Place traps in and around your home to decrease rodent infestation. Clean up any easy-to-get food.
  
Recent research results show that many people who became ill with HPS developed the disease after having been in frequent contact with rodents and/or their droppings around a home or a workplace. On the other hand, many people who became ill reported that they had not seen rodents or rodent droppings at all. Therefore, if you live in an area where the carrier rodents are known to live, try to keep your home, vacation place,

In [145]:
for col, dat in zip(cols, df_medquad.iloc[7000]):
    print(col ,':', dat)

document_id : 0005053
source : GARD
url : https://rarediseases.info.nih.gov/gard/10684/primary-lateral-sclerosis
focus : Primary lateral sclerosis
synonyms : Adult-onset PLS,Adult-onset primary lateral sclerosis
semantic_group : Disorders
pid : 1
qtype : information
question : What is (are) Primary lateral sclerosis ?
answer : Primary lateral sclerosis is a type of motor neuron disease, where nerve cells that control voluntary muscle movement breakdown and die. In primary lateral sclerosis only the upper motor neurons in the brain are affected. Symptoms often begin with problems in the legs (e.g., weakness, stiffness, spasticity, and balance problems), but may also start with hand clumsiness and changes in speech. The symptoms worsen gradually over time, however people with this condition have a normal life expectancy. Progression of symptoms varies from person to person, some people retain the ability to walk without assistance, others eventually require assistive devices such as cane

In [146]:
for col, dat in zip(cols, df_medquad.iloc[500]):
    print(col ,':', dat)

document_id : 0000413
source : GHR
url : https://ghr.nlm.nih.gov/condition/glucose-6-phosphate-dehydrogenase-deficiency
focus : glucose-6-phosphate dehydrogenase deficiency
synonyms : Deficiency of glucose-6-phosphate dehydrogenase,G6PD Deficiency,G6PDD,glucose 6 phosphate dehydrogenase deficiency
semantic_group : Disorders
pid : 1
qtype : information
question : What is (are) glucose-6-phosphate dehydrogenase deficiency ?
answer : Glucose-6-phosphate dehydrogenase deficiency is a genetic disorder that occurs most often in males. This condition mainly affects red blood cells, which carry oxygen from the lungs to tissues throughout the body. In affected individuals, a defect in an enzyme called glucose-6-phosphate dehydrogenase causes red blood cells to break down prematurely. This destruction of red blood cells is called hemolysis.  The most common medical problem associated with glucose-6-phosphate dehydrogenase deficiency is hemolytic anemia, which occurs when red blood cells are dest

## iCLINQ DATA

In [3]:
df = pd.read_csv("../data/raw/icliniq/icliniq_data.csv")
df

,ID,Legacy ID,Title,Author Name,Speciality,Abstract,Affiliation,Publish At,Question,Answer
0,01JF4TEQHFJ5MEKH1QNM9PRT92,897840,What are effective therapies for metastatic br...,Dr. Pawar Satyajit Jalinder,Medical oncology,Metastatic breast cancer occurs when cancer sp...,icliniq,2024-12-15T15:00:05,"Hello doctor,\nMy mother was diagnosed with st...","Hello,\nWelcome to icliniq.com.\nI can underst..."
1,01JF4Q0WT937DQDC2MGR40MTDN,3595800,How does HIV spread?,Dr. Basti Bharatesh Devendra,Dermatology,HIV spreads by certain body fluids from an inf...,icliniq,2024-12-15T14:00:04,"Hello doctor,Last night I went for dinner and ...","Hello,\nWelcome to icliniq.com.\nI read your q..."
2,01JF4KK3NKG5ZRRZ3G0QFXFAB7,3099775,Can recurrent hoarseness without GERD indicate...,Dr. Akshay. B. K.,Otolaryngology (E.N.T),Recurrent hoarseness may result from vocal str...,icliniq,2024-12-15T13:00:04,"Hi doctor,I am a 59-year-old male and a nonsmo...","Hi,\nWelcome to icliniq.com.\nI have read your..."
3,01JF4G52SP8WMFBKZ5V075R066,65337,Is long-term Pantocid-IT use safe?,Dr. Kunal Das,Medical Gastroenterology,Long-term use of Pantocid-IT may cause nutrien...,icliniq,2024-12-15T12:00:04,"Hi doctor,\nI am a 35-year-old male. My height...","Hi,\nWelcome to icliniq.com.\nI have read your..."
4,01JF4CQ76T1P7713XQ099A432S,3876269,Can type 2 diabetes resolve after delivery?,Dr. Nitesh Goyal,Pulmonology (Asthma Doctors),"After giving birth, a person with type 1 diabe...",icliniq,2024-12-15T11:00:04,"Hi doctor,My sister delivered a baby one month...","Hi,\nWelcome to icliniq.com.\nI have gone thro..."
...,...,...,...,...,...,...,...,...,...,...
48452,01HAE1V45T34KDFVW57250NXMG,877873,I am not gaining weight. Please guide me if I ...,Dr. Prakashkumar P Bhatt,Neurology,NaN,icliniq,2018-06-28T10:17:12,"Hi doctor,\nI am 24 years old male. For the pa...","Hello,Welcome to icliniq.com.First of all, che..."
48453,01HAE1JHEGCX40KG1S3BQYYCZ7,793341,Do I need treatment for premature ejaculation?,Dr. Sagar Ramesh Makode,Cardiology,NaN,icliniq,2018-06-20T15:58:21,"Hello doctor,\nI am 37 years old, a gynecologi...","Hello,\nWelcome to icliniq.com.\nNo, it is not..."
48454,01HACE1K8710WS5801P8SKH54E,23914,I had unprotected sex. What are my chances of ...,Dr. Sameer Kumar,Obstetrics and Gynecology,NaN,icliniq,2018-05-15T16:06:35,"Hello doctor,\nI had unprotected sex with my b...","Hi,\nWelcome to icliniq.com.\nPlease do not be..."
48455,01HAE1PJ2TRSHM284PA0K4T1RZ,821390,How many days after HIV do rashes appear?,Dr. Jyotisterna Mittal,Dermatology,NaN,icliniq,2018-04-12T17:12:26,"Hello doctor,\n15 days back, I met a girl and ...","Hello,\nWelcome to icliniq.com.\n\nA rash is a..."


In [4]:
cols2 = df.columns

In [5]:
for col, dat in zip(cols2, df.iloc[15000]):
    print(col ,':', dat)

ID : 01HAE3ZQZ75HK6VFWP1P6JX1RT
Legacy ID : 1218372
Title : How can a white patch under the tongue be treated?
Author Name : Dr. Beryl F
Speciality : Dentistry
Abstract : White patch under the tongue after intake of Losartan may be drug-induced Lichen planus. Consult a specialist to know more.
Affiliation : icliniq
Publish At : 2019-12-07T08:00:17
Question : Hi doctor,
Can you please provide me with your opinion on a white patch under my tongue.  It does not rub off and just noticed it a few days ago. I am currently on Losartan 100 mg.
Answer : Hello,
Welcome to icliniq.com.
I understand your concern and saw your clinical pictures. (attachment removed to protect patient identity).
These pictures look a little blurred and a clearer picture will make the diagnosis easier. It could be a drug-induced lichenoid reaction or lichen planus probably caused by Losartan. Generally, anti-hypertensive drugs such as this are known to cause lichenoid reactions and lichen planus  when medicines are ta

In [157]:
for col, dat in zip(cols2, df.iloc[1000]):
    print(col ,':', dat)

ID : 01HAE2NC3YNHP36BZ5EA8SJBA0
Legacy ID : 1053967
Title : Kindly advise treatment for ichthyosis hystrix.
Author Name : Dr. Rabia Anjum
Speciality : Dermatology
Abstract : Ichthyosis is a family of genetic skin disorders characterized by dry, thickened, scaly skin.
Affiliation : icliniq
Publish At : 2019-05-10T21:42:47
Question : Hello doctor,
I need help. My daughter is 10 years old. And she has ichthyosis hystrix in her body's left section, many doctors said that creams are the only treatment for whole life. Please help.
Answer : Hi,
I am glad you chose icliniq for your medical-related queries.
I understand your concern and will try to help you with it.
Yes, it is ichthyosis hystrix. It is inherited condition, and it is in her genes. Unfortunately, it cannot be reversed by any cream or oral medication. However, it can be improved with oral Retinoids and topical Emollients. The drug of choice is oral Acitretin. Apply Emollients like white soft Paraffin plus liquid Paraffin 4 to 5 ti

In [ ]:
import pandas as pd

def extract_icliniq():
    df = pd.read_csv("../data/raw/icliniq/icliniq_data.csv")
    df = df[['Speciality', 'Title', 'Abstract','Question', 'Answer' ]]
    return df

# if __name__ == '__main__':
#     df_icliniq = extract_icliniq()
#     df_icliniq.to_csv("data/processed/icliniq.csv", index=False)

In [171]:
extract_icliniq()

,Title,Title,Abstract,Question,Answer
0,What are effective therapies for metastatic br...,What are effective therapies for metastatic br...,Metastatic breast cancer occurs when cancer sp...,"Hello doctor,\nMy mother was diagnosed with st...","Hello,\nWelcome to icliniq.com.\nI can underst..."
1,How does HIV spread?,How does HIV spread?,HIV spreads by certain body fluids from an inf...,"Hello doctor,Last night I went for dinner and ...","Hello,\nWelcome to icliniq.com.\nI read your q..."
2,Can recurrent hoarseness without GERD indicate...,Can recurrent hoarseness without GERD indicate...,Recurrent hoarseness may result from vocal str...,"Hi doctor,I am a 59-year-old male and a nonsmo...","Hi,\nWelcome to icliniq.com.\nI have read your..."
3,Is long-term Pantocid-IT use safe?,Is long-term Pantocid-IT use safe?,Long-term use of Pantocid-IT may cause nutrien...,"Hi doctor,\nI am a 35-year-old male. My height...","Hi,\nWelcome to icliniq.com.\nI have read your..."
4,Can type 2 diabetes resolve after delivery?,Can type 2 diabetes resolve after delivery?,"After giving birth, a person with type 1 diabe...","Hi doctor,My sister delivered a baby one month...","Hi,\nWelcome to icliniq.com.\nI have gone thro..."
...,...,...,...,...,...
48452,I am not gaining weight. Please guide me if I ...,I am not gaining weight. Please guide me if I ...,NaN,"Hi doctor,\nI am 24 years old male. For the pa...","Hello,Welcome to icliniq.com.First of all, che..."
48453,Do I need treatment for premature ejaculation?,Do I need treatment for premature ejaculation?,NaN,"Hello doctor,\nI am 37 years old, a gynecologi...","Hello,\nWelcome to icliniq.com.\nNo, it is not..."
48454,I had unprotected sex. What are my chances of ...,I had unprotected sex. What are my chances of ...,NaN,"Hello doctor,\nI had unprotected sex with my b...","Hi,\nWelcome to icliniq.com.\nPlease do not be..."
48455,How many days after HIV do rashes appear?,How many days after HIV do rashes appear?,NaN,"Hello doctor,\n15 days back, I met a girl and ...","Hello,\nWelcome to icliniq.com.\n\nA rash is a..."


## Combine the two sets and miniamal Feature selection